# **Imports**

In [ ]:
import pandas as pd
import zipfile
import gensim
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
import re

# Use the filterwarnings() function to ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Loading Data**

In [ ]:
# Specify the path to the enronsent.zip file
zip_file_path = '/content/drive/MyDrive/enronsent.zip'

# Open the zip file and get the list of file names
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

In [ ]:
# Create an empty string to store the concatenated file contents
concatenated_content_training = ''

# Open the zip file again to read the file contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Iterate through the first 10 files in the file list
    for file_path in file_list[0:11]:
        # Read the file contents as binary data
        with zip_ref.open(file_path) as file:
            file_content = file.read()

        # Decode the file content from bytes to string
        decoded_content = file_content.decode('utf-8', errors='replace')

        # Concatenate the file content to the existing string
        concatenated_content_training += decoded_content
len(concatenated_content_training)


23890105

In [ ]:
# Create an empty string to store the concatenated file contents
concatenated_content_validation = ''

# Open the zip file again to read the file contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Iterate through the first 10 files in the file list
    for file_path in file_list[11:16]:
        # Read the file contents as binary data
        with zip_ref.open(file_path) as file:
            file_content = file.read()

        # Decode the file content from bytes to string
        decoded_content = file_content.decode('utf-8', errors='replace')

        # Concatenate the file content to the existing string
        concatenated_content_validation += decoded_content
len(concatenated_content_validation)

9490330

# **Preprocessing**

In [ ]:
# Remove non-alphabetic characters
cleaned_content_training = re.sub(r'[^a-zA-Z\s]', '', concatenated_content_training)


# Remove URLs
cleaned_content_training = re.sub(r'http\S+|www\S+', '', cleaned_content_training)



In [ ]:
# Remove non-alphabetic characters
cleaned_content_validation = re.sub(r'[^a-zA-Z\s]', '', concatenated_content_validation)

# Remove URLs
cleaned_conten_validation = re.sub(r'http\S+|www\S+', '', cleaned_content_validation)



In [ ]:
# Split the cleaned_content into paragraphs
paragraphs = cleaned_content_training.split("\n\n")

# Convert paragraphs to a list
paragraph_list_training = list(filter(None, paragraphs))
paragraph_list_training=[re.sub('\s+', ' ', paragraph) for paragraph in paragraph_list_training]
len(paragraph_list_training)

128184

In [ ]:
# Split the cleaned_content into paragraphs
paragraphs = cleaned_content_validation.split("\n\n")

# Convert paragraphs to a list
paragraph_list_validation = list(filter(None, paragraphs))
paragraph_list_validation=[re.sub('\s+', ' ', paragraph) for paragraph in paragraph_list_validation]
len(paragraph_list_validation)

63276

In [ ]:
paragraph_list_training[0]

' Attached are two files that illustrate the following'

In [ ]:
paragraph_list_validation[0]

'for how we are going to play these different opportunities off of one another '

In [ ]:
def split_paragraph_into_time_steps(paragraph, time_steps):
    words = paragraph.split()
    data=[]
    for i in range( len(words)-time_steps):
      data.append(words[i:i+time_steps])

    return data

time_steps = 10  # Assuming each time step contains 25 words
num_training_paragraphs = 5000
num_validation_paragraphs = 1000

# Select paragraphs for training
training_paragraphs = paragraph_list_training[:num_training_paragraphs]
training_time_steps = []
for paragraph in training_paragraphs:
    time_steps_list = split_paragraph_into_time_steps(paragraph, time_steps)
    training_time_steps.extend(time_steps_list)

# Select paragraphs for validation
validation_paragraphs = paragraph_list_validation[num_training_paragraphs:num_training_paragraphs+num_validation_paragraphs]
validation_time_steps = []
for paragraph in validation_paragraphs:
    time_steps_list = split_paragraph_into_time_steps(paragraph, time_steps)
    validation_time_steps.extend(time_steps_list)

print(len(training_time_steps))
print(len(validation_time_steps))


74737
13179


In [ ]:
training_time_steps=np.array(training_time_steps)
validation_time_steps=np.array(validation_time_steps)

In [ ]:
print(training_time_steps.shape,validation_time_steps.shape)

(74737, 10) (13179, 10)


# **Word2Vec model**

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
model = Word2Vec(sentences=paragraph_list_training, vector_size=300, window=5, min_count=1, workers=4)

# Save the trained model
model.save("word2vec.model")


In [ ]:
import numpy as np
import keras

# Convert sentences to sequences of word indices
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(training_time_steps.tolist())  # Convert to list before fitting

sequences = tokenizer.texts_to_sequences(training_time_steps.tolist())
max_sequence_length = max(len(seq) for seq in sequences)

# Pad sequences to a fixed length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='pre')

# Split into input (X) and target (y)
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# Convert validation data to sequences
validation_sequences = tokenizer.texts_to_sequences(validation_time_steps.tolist())
validation_padded_sequences = pad_sequences(validation_sequences, maxlen=max_sequence_length, padding='pre')

# Split into validation input (X_val) and target (y_val)
X_val = validation_padded_sequences[:, :-1]
y_val = validation_padded_sequences[:, -1]

# Define the LSTM model
model = keras.Sequential()
model.add(keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length-1))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dense(len(tokenizer.word_index) + 1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X, y, validation_data=(X_val, y_val), epochs=100, batch_size=32)

Epoch 1/100
2336/2336 [==============================] - 33s 12ms/step - loss: 6.7773 - accuracy: 0.0683 - val_loss: 6.3048 - val_accuracy: 0.0813
Epoch 2/100
2336/2336 [==============================] - 16s 7ms/step - loss: 6.0231 - accuracy: 0.1075 - val_loss: 6.1123 - val_accuracy: 0.1052
Epoch 3/100
2336/2336 [==============================] - 16s 7ms/step - loss: 5.4998 - accuracy: 0.1371 - val_loss: 6.1209 - val_accuracy: 0.1076
Epoch 4/100
2336/2336 [==============================] - 16s 7ms/step - loss: 5.0153 - accuracy: 0.1718 - val_loss: 6.1903 - val_accuracy: 0.1125
Epoch 5/100
2336/2336 [==============================] - 15s 6ms/step - loss: 4.5605 - accuracy: 0.2122 - val_loss: 6.3192 - val_accuracy: 0.1086
Epoch 6/100
2336/2336 [==============================] - 15s 7ms/step - loss: 4.1350 - accuracy: 0.2596 - val_loss: 6.4816 - val_accuracy: 0.1084
Epoch 7/100
2336/2336 [==============================] - 15s 7ms/step - loss: 3.7381 - accuracy: 0.3095 - val_loss: 6.6838 

In [ ]:
def predict_next_word(sentence, model, tokenizer):
    sequence = tokenizer.texts_to_sequences([sentence])[0]
    sequence = pad_sequences([sequence], maxlen=max_sequence_length-1, padding='pre')
    predicted_index = np.argmax(model.predict(sequence))
    predicted_word = tokenizer.index_word[predicted_index]
    return predicted_word


In [ ]:
def predict_sentence():
    sentence = input("Enter Next word (-1 to terminate): ")
    predicted_sentence = [sentence]

    while sentence != '-1':
        next_word = predict_next_word(sentence, model, tokenizer)
        answer = input(f"Is your next word: \"{next_word}\"?")
        if answer.lower() == 'yes':
            sentence += ' ' + next_word
            predicted_sentence.append(next_word)
        else:
            sentence = input("Enter next word (-1 to terminate): ")
            if sentence != '-1':
                predicted_sentence.append(sentence)

    final_sentence = ' '.join(predicted_sentence)
    final_sentence = final_sentence.replace(' -1', '')
    print("Your final Sentence is:", final_sentence)

predict_sentence()

Enter Next word (-1 to terminate): my
1/1 [==============================] - 0s 20ms/step
Is your next word: "business"?yes
1/1 [==============================] - 0s 19ms/step
Is your next word: "in"?yes
1/1 [==============================] - 0s 20ms/step
Is your next word: "seguin"?no
Enter next word (-1 to terminate): cairo
1/1 [==============================] - 0s 19ms/step
Is your next word: "in"?no
Enter next word (-1 to terminate): -1
Your final Sentence is: my business in cairo


In [ ]:
def predict_sentence():
    sentence = input("Enter Next word (-1 to terminate): ")
    predicted_sentence = [sentence]

    while sentence != '-1':
        next_word = predict_next_word(sentence, model, tokenizer)
        answer = input(f"Is your next word: \"{next_word}\"?")
        if answer.lower() == 'yes':
            sentence += ' ' + next_word
            predicted_sentence.append(next_word)
        else:
            sentence = input("Enter next word (-1 to terminate): ")
            if sentence != '-1':
                predicted_sentence.append(sentence)

    final_sentence = ' '.join(predicted_sentence)
    final_sentence = final_sentence.replace(' -1', '')
    print("Your final Sentence is:", final_sentence)

predict_sentence()

Enter Next word (-1 to terminate): I
1/1 [==============================] - 0s 31ms/step
Is your next word: "am"?yes
1/1 [==============================] - 0s 20ms/step
Is your next word: "inviting"?yes
1/1 [==============================] - 0s 30ms/step
Is your next word: "less"?no
Enter next word (-1 to terminate): you
1/1 [==============================] - 0s 27ms/step
Is your next word: "in"?yes
1/1 [==============================] - 0s 21ms/step
Is your next word: "california"?yes
1/1 [==============================] - 0s 20ms/step
Is your next word: "power"?no
Enter next word (-1 to terminate): -1
Your final Sentence is: I am inviting you in california


In [ ]:
def predict_sentence():
    sentence = input("Enter Next word (-1 to terminate): ")
    predicted_sentence = [sentence]

    while sentence != '-1':
        next_word = predict_next_word(sentence, model, tokenizer)
        answer = input(f"Is your next word: \"{next_word}\"?")
        if answer.lower() == 'yes':
            sentence += ' ' + next_word
            predicted_sentence.append(next_word)
        else:
            sentence = input("Enter next word (-1 to terminate): ")
            if sentence != '-1':
                predicted_sentence.append(sentence)

    final_sentence = ' '.join(predicted_sentence)
    final_sentence = final_sentence.replace(' -1', '')
    print("Your final Sentence is:", final_sentence)

predict_sentence()

Enter Next word (-1 to terminate): hello
1/1 [==============================] - 0s 28ms/step
Is your next word: "estimated"?no
Enter next word (-1 to terminate): my
1/1 [==============================] - 0s 20ms/step
Is your next word: "business"?no
Enter next word (-1 to terminate): friends
1/1 [==============================] - 0s 19ms/step
Is your next word: "versus"?no
Enter next word (-1 to terminate): -1
Your final Sentence is: hello my friends
